# PDEfind for dataset1 using NNs

In [5]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from aux_for_PDE_find import *
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import AdamW

torch.manual_seed(0)
np.random.seed(0)

class SineActivation(nn.Module):
    def forward(self, x):
        return torch.sin(x)

class MLP(nn.Module):
    def __init__(self, input_dim=2, hidden_dim=32, output_dim=1, num_layers=4):
        super(MLP, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers

        self.input_layer = nn.Linear(input_dim, hidden_dim)
        self.hidden_layers = nn.ModuleList([nn.Linear(hidden_dim, hidden_dim) for _ in range(num_layers - 1)])
        self.output_layer = nn.Linear(hidden_dim, output_dim)

        self.activation = SineActivation()

    def forward(self, inputs):
        out = self.activation(self.input_layer(inputs))
        for layer in self.hidden_layers:
            out = self.activation(layer(out))
        out = self.output_layer(out)
        return out


## Dataset preparation

In [6]:
path_train = "PDEfind_data/1.npz"
data_npz = np.load(path_train)

# Load the data from the .npz file
u = torch.from_numpy(data_npz['u']).type(torch.float32).reshape(-1,1)  # Solution or field
u.requires_grad_()
x = torch.from_numpy(data_npz['x']).type(torch.float32).reshape(-1,1)  # Spatial variable
x.requires_grad_()
t = torch.from_numpy(data_npz['t']).type(torch.float32).reshape(-1,1)   # Temporal variable
t.requires_grad_()

model = MLP()

In [7]:

batch_size = 512

training_set = DataLoader(TensorDataset(u,x,t), batch_size=batch_size, shuffle=True)


learning_rate = 0.01
epochs = 200
step_size = 50
gamma = 0.5
optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

l = torch.nn.MSELoss()
freq_print = 1
for epoch in range(epochs):
    train_mse = 0.0
    for step, (u_train,x_train,t_train) in enumerate(training_set):
        inputs = torch.cat([x_train,t_train],dim=1)
        optimizer.zero_grad()
        u_pred = model(inputs)
        loss_f = l(u_pred, u_train)
        loss_f.backward()
        optimizer.step()
        train_mse += loss_f.item()
    train_mse /= len(training_set)

    scheduler.step()

    if epoch % freq_print == 0:
        model.eval()
        with torch.no_grad():
            input_nn = torch.cat([x,t],dim=1)
            u_nn = model(input_nn)
            error = (torch.norm(u-u_nn,p=2)/torch.norm(u,p=2)).item()
            print("######### Epoch:", epoch, " ######### Train Loss (%):", train_mse*100, " ######### relative L2 (%):", error*100)




######### Epoch: 0  ######### Train Loss (%): 1.239252305480049  ######### relative L2 (%): 23.668865859508514
######### Epoch: 1  ######### Train Loss (%): 0.10352173521110386  ######### relative L2 (%): 10.14656350016594
######### Epoch: 2  ######### Train Loss (%): 0.027100370061816648  ######### relative L2 (%): 5.46284057199955
######### Epoch: 3  ######### Train Loss (%): 0.010375942784936771  ######### relative L2 (%): 4.269769415259361
######### Epoch: 4  ######### Train Loss (%): 0.009638901789074618  ######### relative L2 (%): 3.9758604019880295
######### Epoch: 5  ######### Train Loss (%): 0.014721621946413434  ######### relative L2 (%): 4.321496933698654
######### Epoch: 6  ######### Train Loss (%): 0.006656369563036908  ######### relative L2 (%): 3.2600797712802887
######### Epoch: 7  ######### Train Loss (%): 0.004711198380391579  ######### relative L2 (%): 3.0229421332478523
######### Epoch: 8  ######### Train Loss (%): 0.005515548635979512  ######### relative L2 (%): 3.

In [8]:
# Save the model parameters
#torch.save(model.state_dict(), "model_params_PDE1NN.pth")
# Load the parameters into the model
model.load_state_dict(torch.load("model_params_PDE1NN.pth"))

/var/folders/2c/01bhfspx2hj56fb5ryg654lr0000gn/T/ipykernel_25333/2949376126.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model_param

<All keys matched successfully>

## Build of $\Theta$

In [9]:
x = x.clone().detach()
t = t.clone().detach()

input = torch.cat([x,t],dim=1)
input.requires_grad = True
u_nn = model(input)

#compute the derivatives
grad_u = torch.autograd.grad(outputs = u_nn, inputs=input, grad_outputs=torch.ones_like(u_nn), create_graph=True, retain_graph=True)[0]
dudt = grad_u[:, 1].unsqueeze(-1)
dudx = grad_u[:, 0].unsqueeze(-1)
dudx_2 = torch.autograd.grad(dudx.sum(), input, create_graph=True, retain_graph=True)[0][:, 0].unsqueeze(-1)
dudx_3 = torch.autograd.grad(dudx_2.sum(), input, create_graph=True, retain_graph=True)[0][:, 0].unsqueeze(-1)

# Compute other terms
u2 = torch.pow(u, 2)
u3 = torch.pow(u, 3)  

# Compute mixed terms
u_times_dudx = torch.mul(u, dudx)
u2_times_dudx = torch.mul(u2, dudx)
u3_times_dudx = torch.mul(u3, dudx)

u_times_dudx_2 = torch.mul(u, dudx_2)
u2_times_dudx_2 = torch.mul(u2, dudx_2)
u3_times_dudx_2 = torch.mul(u3, dudx_2)

u_times_dudx_3 = torch.mul(u, dudx_3)
u2_times_dudx_3 = torch.mul(u2, dudx_3)
u3_times_dudx_3 = torch.mul(u3, dudx_3)

# Compute bias
bias = torch.ones_like(u_nn)


# Combine all vectors into a single matrix
Theta = torch.cat([
    bias,               # 0
    u,                  # 1
    dudx,               # 3
    dudx_2,             # 4
    dudx_3,             # 5
    u2,                 # 6
    u3,                 # 7
    u_times_dudx,       # 8
    u2_times_dudx,      # 9
    u3_times_dudx,      # 10
    u_times_dudx_2,     # 11
    u2_times_dudx_2,    # 12
    u3_times_dudx_2,    # 13
    u_times_dudx_3,     # 14
    u2_times_dudx_3,    # 15
    u3_times_dudx_3     # 16
], dim=1)

# Update Theta names
Theta_names = [
    "bias",             # 0
    "u",                # 1
    "dudx",             # 3
    "dudx_2",           # 4
    "dudx_3",           # 5
    "u2",               # 6
    "u3",               # 7
    "u_times_dudx",     # 8
    "u2_times_dudx",    # 9
    "u3_times_dudx",    # 10
    "u_times_dudx_2",   # 11
    "u2_times_dudx_2",  # 12
    "u3_times_dudx_2",  # 13
    "u_times_dudx_3",   # 14
    "u2_times_dudx_3",  # 15
    "u3_times_dudx_3"   # 16
]


## Regression and printing of the PDE

In [10]:
alpha = [0.1,0.01,0.0]
threshold=4
Theta_= Theta.detach().numpy()
dudt_ = dudt.detach().numpy()
selected_names, regression_coef_ = myregression2(Theta_,dudt_,Theta_names,threshold=threshold,alpha=alpha)

Regression # 1
dudx_2  c=  0.09624757
u_times_dudx  c=  -0.9050635
Mean Squared Error: 3.6148529034107924e-05
###################### end of regression 1 ##############################

Regression # 2
dudx_2  c=  0.09851861
u_times_dudx  c=  -0.99238396
Mean Squared Error: 5.575024260906503e-06
###################### end of regression 2 ##############################

Regression # 3
dudx_2  c=  0.09852514
u_times_dudx  c=  -0.992496
Mean Squared Error: 5.573814178205794e-06
###################### end of regression 3 ##############################



In [11]:
printPDE(selected_names, regression_coef_)

dudt = 0.09852514 * dudx_2 +
       -0.992496 * u_times_dudx
